In [1]:
import time
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import drizzlepac
import grizli
import glob
from grizli import utils
import importlib
from grizli.prep import process_direct_grism_visit
from hsaquery import query, overlaps
from grizli.pipeline import auto_script
from grizli.multifit import GroupFLT, MultiBeam, get_redshift_fit_defaults
import os
from grizli.pipeline import photoz
from astropy.table import Table
import eazy
from joblib import Parallel, delayed

The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       imagefindpars           mapreg              photeq       
     pixreplace           pixtopix            pixtosky        refimagefindpars  
     resetbits          runastrodriz          skytopix           tweakback      
      tweakreg           updatenpol


In [2]:
field           = 'GS1'
ref_filter      = 'F105W'
HOME_PATH       = '/Users/rsimons/Desktop/clear/for_hackday/%s'%field
PATH_TO_SCRIPTS = '/Users/rsimons/Desktop/git/clear_local/example_notebooks'
PATH_TO_CATS    = '/Users/rsimons/Desktop/clear/Catalogs'
PATH_TO_RAW     = glob.glob(HOME_PATH + '/*/RAW')[0]
PATH_TO_PREP    = glob.glob(HOME_PATH + '/*/PREP')[0]


class Pointing():
    """ Generalization of GN1, GS1, ERSPRIME, etc

    To change field-dependent catalog, seg map, ref image, and padding
    only need to change them here.

    """
    def __init__(self, field, ref_filter):
        if 'N' in field.upper():
            self.pad = 500 # really only necessary for GDN
            self.radec_catalog = PATH_TO_CATS + '/old_radeccats/goodsN_radec.cat'
            self.seg_map =  PATH_TO_CATS + '/Goods_N_plus_seg.fits'
            self.catalog =  PATH_TO_CATS + '/goodsn-F105W-astrodrizzle-v4.4_drz_sub_plus.cat'
            self.ref_image =  PATH_TO_CATS + '/goodsn-F105W-astrodrizzle-v4.4_drz_sci.fits'
            
            self.tempfilt, self.coeffs, self.temp_sed, self.pz = readEazyBinary(MAIN_OUTPUT_FILE='goodsn_3dhst.v4.1', OUTPUT_DIRECTORY=PATH_TO_CATS, CACHE_FILE='Same')
            self.params = {}
            self.params['CATALOG_FILE'] = PATH_TO_CATS + '/{0}_3dhst.{1}.cats/Catalog/{0}_3dhst.{1}.cat'.format('goodsn', 'v4.1')
            self.params['Z_STEP'] = 0.002
            self.params['Z_MAX'] = 4
            self.params['MAIN_OUTPUT_FILE'] = '{0}_3dhst.{1}.eazypy'.format('goodsn', 'v4.1')
            self.params['PRIOR_FILTER'] = 205
            self.params['MW_EBV'] = {'aegis':0.0066, 'cosmos':0.0148, 'goodss':0.0069, 
                                    'uds':0.0195, 'goodsn':0.0103}['goodsn']
            self.params['TEMPLATES_FILE'] = 'templates/fsps_full/tweak_fsps_QSF_12_v3.param'
            self.translate_file = PATH_TO_CATS + '/{0}_3dhst.{1}.cats/Eazy/{0}_3dhst.{1}.translate'.format('goodsn', 'v4.1')

        elif 'S' in field.upper():
            self.pad = 200 # grizli default
            self.radec_catalog =  PATH_TO_CATS + '/old_radeccats/goodsS_radec.cat'
            self.seg_map =  PATH_TO_CATS + '/Goods_S_plus_seg.fits'
            self.catalog =  PATH_TO_CATS + '/goodss-F105W-astrodrizzle-v4.3_drz_sub_plus.cat'
            self.ref_image =  PATH_TO_CATS + '/goodss-F105W-astrodrizzle-v4.3_drz_sci.fits'


            self.tempfilt, self.coeffs, self.temp_sed, self.pz = readEazyBinary(MAIN_OUTPUT_FILE='goodss_3dhst.v4.1', OUTPUT_DIRECTORY=PATH_TO_CATS, CACHE_FILE='Same')
            self.params = {}
            self.params['CATALOG_FILE'] = PATH_TO_CATS + '/{0}_3dhst.{1}.cats/Catalog/{0}_3dhst.{1}.cat'.format('goodss', 'v4.1')
            self.params['Z_STEP'] = 0.002
            self.params['Z_MAX'] = 4
            self.params['MAIN_OUTPUT_FILE'] = '{0}_3dhst.{1}.eazypy'.format('goodss', 'v4.1')
            self.params['PRIOR_FILTER'] = 205
            self.params['MW_EBV'] = {'aegis':0.0066, 'cosmos':0.0148, 'goodss':0.0069, 
                                    'uds':0.0195, 'goodsn':0.0103}['goodss']
            self.params['TEMPLATES_FILE'] = 'templates/fsps_full/tweak_fsps_QSF_12_v3.param'
            self.translate_file = PATH_TO_CATS + '/{0}_3dhst.{1}.cats/Eazy/{0}_3dhst.{1}.translate'.format('goodss', 'v4.1')




def readEazyBinary(MAIN_OUTPUT_FILE='photz', OUTPUT_DIRECTORY='./OUTPUT', CACHE_FILE='Same'):

    """
    Author: Gabe Brammer
    This function has been clipped from eazyPy.py in thethreedhst git respository
    https://github.com/gbrammer/threedhst/tree/master/threedhst

    tempfilt, coeffs, temp_sed, pz = readEazyBinary(MAIN_OUTPUT_FILE='photz', \
                                                OUTPUT_DIRECTORY='./OUTPUT', \
                                                CACHE_FILE = 'Same')

    Read Eazy BINARY_OUTPUTS files into structure data.
    
    If the BINARY_OUTPUTS files are not in './OUTPUT', provide either a relative or absolute path
    in the OUTPUT_DIRECTORY keyword.
    
    By default assumes that CACHE_FILE is MAIN_OUTPUT_FILE+'.tempfilt'.
    Specify the full filename if otherwise. 
    """
    
    #root='COSMOS/OUTPUT/cat3.4_default_lines_zp33sspNoU'
    
    root = OUTPUT_DIRECTORY+'/'+MAIN_OUTPUT_FILE
    
    ###### .tempfilt
    if CACHE_FILE == 'Same':
        CACHE_FILE = root+'.tempfilt'
    
    if os.path.exists(CACHE_FILE) is False:
        print(('File, %s, not found.' %(CACHE_FILE)))
        return -1,-1,-1,-1
    
    f = open(CACHE_FILE,'rb')
    
    s = np.fromfile(file=f,dtype=np.int32, count=4)
    NFILT=s[0]
    NTEMP=s[1]
    NZ=s[2]
    NOBJ=s[3]
    tempfilt = np.fromfile(file=f,dtype=np.double,count=NFILT*NTEMP*NZ).reshape((NZ,NTEMP,NFILT)).transpose()
    lc = np.fromfile(file=f,dtype=np.double,count=NFILT)
    zgrid = np.fromfile(file=f,dtype=np.double,count=NZ)
    fnu = np.fromfile(file=f,dtype=np.double,count=NFILT*NOBJ).reshape((NOBJ,NFILT)).transpose()
    efnu = np.fromfile(file=f,dtype=np.double,count=NFILT*NOBJ).reshape((NOBJ,NFILT)).transpose()
    
    f.close()
    
    tempfilt  = {'NFILT':NFILT,'NTEMP':NTEMP,'NZ':NZ,'NOBJ':NOBJ,\
                 'tempfilt':tempfilt,'lc':lc,'zgrid':zgrid,'fnu':fnu,'efnu':efnu}
    
    ###### .coeff
    f = open(root+'.coeff','rb')
    
    s = np.fromfile(file=f,dtype=np.int32, count=4)
    NFILT=s[0]
    NTEMP=s[1]
    NZ=s[2]
    NOBJ=s[3]
    coeffs = np.fromfile(file=f,dtype=np.double,count=NTEMP*NOBJ).reshape((NOBJ,NTEMP)).transpose()
    izbest = np.fromfile(file=f,dtype=np.int32,count=NOBJ)
    tnorm = np.fromfile(file=f,dtype=np.double,count=NTEMP)
    
    f.close()
    
    coeffs = {'NFILT':NFILT,'NTEMP':NTEMP,'NZ':NZ,'NOBJ':NOBJ,\
              'coeffs':coeffs,'izbest':izbest,'tnorm':tnorm}
              
    ###### .temp_sed
    f = open(root+'.temp_sed','rb')
    s = np.fromfile(file=f,dtype=np.int32, count=3)
    NTEMP=s[0]
    NTEMPL=s[1]
    NZ=s[2]
    templam = np.fromfile(file=f,dtype=np.double,count=NTEMPL)
    temp_seds = np.fromfile(file=f,dtype=np.double,count=NTEMPL*NTEMP).reshape((NTEMP,NTEMPL)).transpose()
    da = np.fromfile(file=f,dtype=np.double,count=NZ)
    db = np.fromfile(file=f,dtype=np.double,count=NZ)
    
    f.close()
    
    temp_sed = {'NTEMP':NTEMP,'NTEMPL':NTEMPL,'NZ':NZ,\
              'templam':templam,'temp_seds':temp_seds,'da':da,'db':db}
              
    ###### .pz
    if os.path.exists(root+'.pz'):
        f = open(root+'.pz','rb')
        s = np.fromfile(file=f,dtype=np.int32, count=2)
        NZ=s[0]
        NOBJ=s[1]
        chi2fit = np.fromfile(file=f,dtype=np.double,count=NZ*NOBJ).reshape((NOBJ,NZ)).transpose()

        ### This will break if APPLY_PRIOR No
        s = np.fromfile(file=f,dtype=np.int32, count=1)
        
        if len(s) > 0:
            NK = s[0]
            kbins = np.fromfile(file=f,dtype=np.double,count=NK)
            priorzk = np.fromfile(file=f, dtype=np.double, count=NZ*NK).reshape((NK,NZ)).transpose()
            kidx = np.fromfile(file=f,dtype=np.int32,count=NOBJ)
            pz = {'NZ':NZ,'NOBJ':NOBJ,'NK':NK, 'chi2fit':chi2fit, 'kbins':kbins, 'priorzk':priorzk,'kidx':kidx}
        else:
            pz = None
        
        f.close()
        
    else:
        pz = None
    
    if False:
        f = open(root+'.zbin','rb')
        s = np.fromfile(file=f,dtype=np.int32, count=1)
        NOBJ=s[0]
        z_a = np.fromfile(file=f,dtype=np.double,count=NOBJ)
        z_p = np.fromfile(file=f,dtype=np.double,count=NOBJ)
        z_m1 = np.fromfile(file=f,dtype=np.double,count=NOBJ)
        z_m2 = np.fromfile(file=f,dtype=np.double,count=NOBJ)
        z_peak = np.fromfile(file=f,dtype=np.double,count=NOBJ)
        f.close()
        
    ###### Done.    
    return tempfilt, coeffs, temp_sed, pz



In [3]:
os.chdir(PATH_TO_PREP)
files = glob.glob('%s/*flt.fits'%PATH_TO_RAW)
info = grizli.utils.get_flt_info(files)
visits, filters = grizli.utils.parse_flt_files(info=info, uniquename=True)


goodss-01-bhj-01-177.0-F140W 4
goodss-01-bhj-01-177.0-G141 4
GS1-cxt-09-227.0-F105W 2
GS1-cxt-10-227.0-F105W 2
GS1-cxt-07-249.0-F105W 2
GS1-cxt-08-249.0-F105W 2
GS1-cxt-11-269.0-F105W 2
GS1-cxt-12-269.0-F105W 2
GS1-cxt-09-227.0-G102 4
GS1-cxt-10-227.0-G102 4
GS1-cxt-07-249.0-G102 4
GS1-cxt-08-249.0-G102 4
GS1-cxt-11-269.0-G102 4
GS1-cxt-12-269.0-G102 4


In [4]:
def grizli_model(visits, field = 'GN2', ref_filter_1 = 'F105W', ref_grism_1 = 'G102', ref_filter_2 = 'F140W', ref_grism_2 = 'G141', load_only = True, mag_lim = 25, include_G141 = True):

    all_grism_files = []
    all_direct_files = []
    product_names = np.array([visit['product'] for visit in visits])
    filter_names = np.array([visit['product'].split('-')[-1] for visit in visits])
    basenames = np.array([visit['product'].split('.')[0]+'.0' for visit in visits])

    for v, visit in enumerate(visits):
        product = product_names[v]
        basename = basenames[v]
        filt1 = filter_names[v]        
        #filter_name = visit['product'].split('-')[-1]
        field_in_contest = visit['product'].split('-')[0].upper()
        #if field_in_contest.upper() != 'GOODSS':
        #    print (field_in_contest.upper())
            #if field_in_contest == field or field_in_contest in overlapping_fields[field]:
        if (ref_filter_1.lower() in filt1) or (ref_filter_2.lower() in filt1):
            #Find grism files with a direct image
            all_direct_files.extend(visit['files'])
            grism_index_1 = np.where((basenames == basename) & (filter_names == ref_grism_1.lower()))[0]

            if len(grism_index_1) > 0:
                all_grism_files.extend(visits[grism_index_1[0]]['files'])
                print(filter_names[grism_index_1[0]], visits[grism_index_1[0]]['product'])
            if include_G141:
                #Include G141 observatinos
                grism_index_2= np.where((basenames == basename) & (filter_names == ref_grism_2.lower()))[0]

                if len(grism_index_2) > 0:
                    all_grism_files.extend(visits[grism_index_2[0]]['files'])
                    print(filter_names[grism_index_2[0]], visits[grism_index_2[0]]['product'])

    print (all_direct_files, all_grism_files)

    p = Pointing(field=field, ref_filter=ref_filter_1)
    if load_only:
        print('Loading contamination models...')

    grp = GroupFLT(
        grism_files=all_grism_files, 
        direct_files=[], 
        ref_file = p.ref_image,
        seg_file = p.seg_map,
        catalog  = p.catalog,
        pad=p.pad,
        cpu_count=8)

    if not load_only:
        print('Computing contamination models...')
        grp.compute_full_model(mag_limit=mag_lim)
        print('Refining List..')
        grp.refine_list(poly_order=2, mag_limits=[16, 24], verbose=False)
        print('Saving contamination models')
        grp.save_full_data()
    return grp


mag_lim = 21

grp = grizli_model(visits, field = field, ref_filter_1 = 'F105W', ref_grism_1 = 'G102', ref_filter_2 = 'F140W', ref_grism_2 = 'G141',
                   load_only = True, mag_lim = mag_lim)


g141 goodss-01-bhj-01-177.0-g141
g102 gs1-cxt-09-227.0-g102
g102 gs1-cxt-10-227.0-g102
g102 gs1-cxt-07-249.0-g102
g102 gs1-cxt-08-249.0-g102
g102 gs1-cxt-11-269.0-g102
g102 gs1-cxt-12-269.0-g102
['ibhj01ioq_flt.fits', 'ibhj01ivq_flt.fits', 'ibhj01j2q_flt.fits', 'ibhj01j9q_flt.fits', 'icxt09ezq_flt.fits', 'icxt09f4q_flt.fits', 'icxt10daq_flt.fits', 'icxt10dhq_flt.fits', 'icxt07dwq_flt.fits', 'icxt07e1q_flt.fits', 'icxt08e6q_flt.fits', 'icxt08ebq_flt.fits', 'icxt11qhq_flt.fits', 'icxt11qmq_flt.fits', 'icxt12quq_flt.fits', 'icxt12qzq_flt.fits'] ['ibhj01iqq_flt.fits', 'ibhj01ixq_flt.fits', 'ibhj01j4q_flt.fits', 'ibhj01jbq_flt.fits', 'icxt09f0q_flt.fits', 'icxt09f2q_flt.fits', 'icxt09f5q_flt.fits', 'icxt09f7q_flt.fits', 'icxt10dbq_flt.fits', 'icxt10deq_flt.fits', 'icxt10diq_flt.fits', 'icxt10dkq_flt.fits', 'icxt07dxq_flt.fits', 'icxt07dzq_flt.fits', 'icxt07e2q_flt.fits', 'icxt07e4q_flt.fits', 'icxt08e7q_flt.fits', 'icxt08e9q_flt.fits', 'icxt08ecq_flt.fits', 'icxt08eeq_flt.fits', 'icxt11qiq_

In [5]:
p = Pointing(field = field, ref_filter = ref_filter)
eazy.symlink_eazy_inputs(path=os.path.dirname(eazy.__file__)+'/data', 
             path_is_env=False)
templ0 = grizli.utils.load_templates(fwhm=1200, line_complexes=True, stars=False, 
                                     full_line_list=None,  continuum_list=None, 
                                     fsps_templates=True)

# Load individual line templates for fitting the line fluxes
templ1 = grizli.utils.load_templates(fwhm=1200, line_complexes=False, stars=False, 
                                     full_line_list=None, continuum_list=None, 
                                     fsps_templates=True)

pline = {'kernel': 'point', 'pixfrac': 0.2, 'pixscale': 0.1, 'size': 8, 'wcs': None}
ez = eazy.photoz.PhotoZ(param_file=None, translate_file=p.translate_file, 
                        zeropoint_file=None, params=p.params, 
                        load_prior=True, load_products=False)

ep = photoz.EazyPhot(ez, grizli_templates=templ0, zgrid=ez.zgrid)







File, /Users/rsimons/Desktop/clear/Catalogs/goodss_3dhst.v4.1.tempfilt, not found.
/anaconda2/envs/grizli/lib/python3.5/site-packages/eazy/data/templates -> ./templates
/anaconda2/envs/grizli/lib/python3.5/site-packages/eazy/data/filters/FILTER.RES.latest -> ./FILTER.RES.latest
Read default param file: /anaconda2/envs/grizli/lib/python3.5/site-packages/eazy/data/zphot.param.default
Read CATALOG_FILE: /Users/rsimons/Desktop/clear/Catalogs/goodss_3dhst.v4.1.cats/Catalog/goodss_3dhst.v4.1.cat
f_F160W e_F160W (205): hst/wfc3/IR/f160w.dat
f_U e_U (103): ESO/vimos_u.res
f_F435W e_F435W (  1): hst/ACS_update_sep07/wfc_f435w_t77.dat
f_F606Wcand e_F606Wcand (236): hst/ACS_update_sep07/wfc_f606w_t81.dat
f_F606W e_F606W (  4): hst/ACS_update_sep07/wfc_f606w_t77.dat
f_R e_R (260): ESO/VIMOS/R.dat
f_F775W e_F775W (  5): hst/ACS_update_sep07/wfc_f775w_t77.dat
f_F814Wcand e_F814Wcand (239): hst/ACS_update_sep07/wfc_f814w_t81.dat
f_F850LP e_F850LP (  7): hst/ACS_update_sep07/wfc_f850lp_t77.dat
f_F850L

In [15]:
def grizli_fit(grp, id_fit, field = '', ref_filter = 'F105W', use_pz_prior = True, use_phot = True, scale_phot = True, templ0 = None, templ1 = None, ez = None, ep = None, pline = None):
    beams = grp.get_beams(id_fit, size=80) #size??
    if beams != []:
        print("beams: ", beams)
        mb = grizli.multifit.MultiBeam(beams, fcontam=1.0, group_name=field)
        mb.write_master_fits()

        # Fit polynomial model for initial continuum subtraction
        wave = np.linspace(2000,2.5e4,100)
        poly_templates = grizli.utils.polynomial_templates(
            wave=wave, 
            order=7,
            line=False)

        pfit = mb.template_at_z(
            z=0, 
            templates=poly_templates, 
            fit_background=True, 
            fitter='lstsq', 
            fwhm=1400, 
            get_uncertainties=2)


        if pfit != None:
        # Drizzle grisms / PAs
            hdu, fig = mb.drizzle_grisms_and_PAs(
                size=32, 
                fcontam=0.2, 
                flambda=False, 
                scale=1, 
                pixfrac=0.5, 
                kernel='point', 
                make_figure=True, 
                usewcs=False, 
                zfit=pfit,
                diff=True)
            # Save drizzled ("stacked") 2D trace as PNG and FITS
            fig.savefig('{0}_{1:05d}.stack.png'.format(field, id_fit))
            hdu.writeto('{0}_{1:05d}.stack.fits'.format(field, id_fit), clobber=True)



            if use_pz_prior:
                #use redshift prior from z_phot
                prior = np.zeros((2, len(p.tempfilt['zgrid'])))
                prior[0] = p.tempfilt['zgrid']
                prior[1] = p.pz['chi2fit'][:,id]
            else:
                prior = None 
            order = 0



            tab = utils.GTable()
            tab['ra'] = [mb.ra]
            tab['dec'] = [mb.dec]

            tab['id'] = id_fit
            phot, ii, dd = ep.get_phot_dict(tab['ra'][0], tab['dec'][0])

            out = grizli.fitting.run_all(
                id_fit, 
                t0=templ0, 
                t1=templ1, 
                fwhm=1200, 
                zr=[0.0, 3.5], 
                dz=[0.004, 0.0005], 
                fitter='nnls',
                group_name=field,
                fit_stacks=True, 
                prior=None, 
                fcontam=0.,
                pline=pline, 
                mask_sn_limit=7, 
                fit_only_beams=False,
                fit_beams=True, 
                root=field,
                fit_trace_shift=False, 
                phot=phot, 
                verbose=True, 
                scale_photometry=order, 
                show_beams=True)
            mb, st, fit, tfit, line_hdu = out
            fit_hdu = fits.open('{0}_{1:05d}.full.fits'.format(field, id_fit)) 

            fit_hdu.info()
            # same as the fit table above, redshift fit to the stacked spectra
            fit_stack = Table(fit_hdu['ZFIT_STACK'].data) 


            # zoom in around the initial best-guess with the individual "beam" spectra
            fit_beam = Table(fit_hdu['ZFIT_BEAM'].data)   

            templ = Table(fit_hdu['TEMPL'].data)
            print('{0} has lines [{1}]'.format(fit_hdu.filename(), fit_hdu[0].header['HASLINES']))

            # Helper script for plotting them, not generated automatically
            fig = grizli.fitting.show_drizzled_lines(fit_hdu, size_arcsec=1.6, cmap='plasma_r')
            fig.savefig('{0}_{1:05d}.line.png'.format(field, id_fit))
            plt.close('all')


In [ ]:
good = np.where(np.array(grp.catalog['MAG_AUTO']) < 22)[0]
for g in good:
    id_fit = np.array(grp.catalog['NUMBER'])[g]
    mag_fit = grp.catalog['MAG_AUTO'][g]
    grizli_fit(grp, id_fit = id_fit, field = field,
               use_pz_prior = False, use_phot = True, scale_phot = True,
               templ0 = templ0, templ1 = templ1, ez = ez, ep = ep, pline = pline,)
        


        

'''
grizli_fit(grp, id_fit = id_fit, field = field,
           use_pz_prior = False, use_phot = True, scale_phot = True,
           templ0 = templ0, templ1 = templ1, ez = ez, ep = ep, pline = pline,)
'''

beams:  [<grizli.model.BeamCutout object at 0x1df0d597b8>, <grizli.model.BeamCutout object at 0x1df0d59898>, <grizli.model.BeamCutout object at 0x1df0e15cf8>, <grizli.model.BeamCutout object at 0x1df0e2ae10>, <grizli.model.BeamCutout object at 0x1df0e41c50>, <grizli.model.BeamCutout object at 0x1df4b88f98>, <grizli.model.BeamCutout object at 0x1df4ba1dd8>, <grizli.model.BeamCutout object at 0x1df4bafe80>, <grizli.model.BeamCutout object at 0x1df0532cc0>, <grizli.model.BeamCutout object at 0x1df053fc50>, <grizli.model.BeamCutout object at 0x1df055deb8>, <grizli.model.BeamCutout object at 0x1df0d28b38>, <grizli.model.BeamCutout object at 0x1df0d36908>, <grizli.model.BeamCutout object at 0x1df0d44f98>, <grizli.model.BeamCutout object at 0x1df0cbfdd8>, <grizli.model.BeamCutout object at 0x1df0cabe80>, <grizli.model.BeamCutout object at 0x1df0cc9cc0>, <grizli.model.BeamCutout object at 0x1df4d6dc50>, <grizli.model.BeamCutout object at 0x1df4d44eb8>, <grizli.model.BeamCutout object at 0x1df4

  0.7718    5293.0 (0.1924) 144/377
  0.7789    5282.7 (0.1924) 145/377
  0.7860    5271.5 (0.1924) 146/377
  0.7932    5271.5 (0.1924) 147/377
  0.8004    5272.3 (0.1924) 148/377
  0.8076    5246.8 (0.1924) 149/377
  0.8148    5206.8 (0.1924) 150/377
  0.8221    5171.6 (0.1924) 151/377
  0.8294    5137.7 (0.1924) 152/377
  0.8368    5098.8 (0.1924) 153/377
  0.8441    5058.0 (0.8441) 154/377
  0.8515    5017.3 (0.8515) 155/377
  0.8589    4977.0 (0.8589) 156/377
  0.8664    4960.6 (0.8664) 157/377
  0.8739    4944.1 (0.8739) 158/377
  0.8814    4922.7 (0.8814) 159/377
  0.8889    4901.7 (0.8889) 160/377
  0.8965    4876.8 (0.8965) 161/377
  0.9041    4848.8 (0.9041) 162/377
  0.9117    4812.9 (0.9117) 163/377
  0.9194    4758.8 (0.9194) 164/377
  0.9271    4707.3 (0.9271) 165/377
  0.9348    4671.2 (0.9348) 166/377
  0.9425    4615.0 (0.9425) 167/377
  0.9503    4516.2 (0.9503) 168/377
  0.9581    4370.8 (0.9581) 169/377
  0.9660    4188.6 (0.9660) 170/377
  0.9739    4207.9 (0.9660) 

  2.8190   16410.3 (0.9660) 336/377
  2.8344   16443.1 (0.9660) 337/377
  2.8497   16481.2 (0.9660) 338/377
  2.8651   16548.5 (0.9660) 339/377
  2.8806   16637.6 (0.9660) 340/377
  2.8962   16719.1 (0.9660) 341/377
  2.9118   16830.5 (0.9660) 342/377
  2.9275   16940.0 (0.9660) 343/377
  2.9432   17022.6 (0.9660) 344/377
  2.9590   17058.9 (0.9660) 345/377
  2.9749   17082.8 (0.9660) 346/377
  2.9908   17121.6 (0.9660) 347/377
  3.0068   17148.9 (0.9660) 348/377
  3.0229   17175.5 (0.9660) 349/377
  3.0390   17149.1 (0.9660) 350/377
  3.0552   17092.6 (0.9660) 351/377
  3.0715   17068.3 (0.9660) 352/377
  3.0878   17004.4 (0.9660) 353/377
  3.1042   16891.0 (0.9660) 354/377
  3.1206   16778.4 (0.9660) 355/377
  3.1371   16653.1 (0.9660) 356/377
  3.1537   16512.0 (0.9660) 357/377
  3.1704   16366.9 (0.9660) 358/377
  3.1871   16191.6 (0.9660) 359/377
  3.2038   16054.2 (0.9660) 360/377
  3.2207   15952.5 (0.9660) 361/377
  3.2376   15886.6 (0.9660) 362/377
  3.2546   15842.7 (0.9660) 

GS1_44999.beams.fits
1 ibhj01iqq_flt.fits G141
2 ibhj01ixq_flt.fits G141
3 ibhj01j4q_flt.fits G141
4 ibhj01jbq_flt.fits G141
5 icxt09f0q_flt.fits G102
6 icxt09f2q_flt.fits G102
7 icxt09f5q_flt.fits G102
8 icxt09f7q_flt.fits G102
9 icxt10dbq_flt.fits G102
10 icxt10deq_flt.fits G102
11 icxt10diq_flt.fits G102
12 icxt10dkq_flt.fits G102
13 icxt07dxq_flt.fits G102
14 icxt07dzq_flt.fits G102
15 icxt07e2q_flt.fits G102
16 icxt07e4q_flt.fits G102
17 icxt08e7q_flt.fits G102
18 icxt08e9q_flt.fits G102
19 icxt08ecq_flt.fits G102
20 icxt08eeq_flt.fits G102
21 icxt11qiq_flt.fits G102
22 icxt11qkq_flt.fits G102
23 icxt11qnq_flt.fits G102
24 icxt11qpq_flt.fits G102
25 icxt12qvq_flt.fits G102
26 icxt12qxq_flt.fits G102
27 icxt12r0q_flt.fits G102
28 icxt12r2q_flt.fits G102
Load file GS1_44999.stack.fits
  G102 G141
User templates! N=16 

  0.0000   13260.9 (0.0000) 1/377
  0.0040   13258.3 (0.0040) 2/377
  0.0080   13255.2 (0.0080) 3/377
  0.0121    7504.9 (0.0121) 4/377
  0.0161    7481.2 (0.0161) 5/

  1.0138    6719.1 (1.0138) 176/377
  1.0218    6690.9 (1.0218) 177/377
  1.0299    6659.3 (1.0299) 178/377
  1.0381    6632.6 (1.0381) 179/377
  1.0462    6622.7 (1.0462) 180/377
  1.0544    6626.5 (1.0462) 181/377
  1.0627    6635.4 (1.0462) 182/377
  1.0709    6652.7 (1.0462) 183/377
  1.0792    6671.3 (1.0462) 184/377
  1.0876    6682.3 (1.0462) 185/377
  1.0959    6689.0 (1.0462) 186/377
  1.1043    6694.9 (1.0462) 187/377
  1.1128    6697.1 (1.0462) 188/377
  1.1212    6696.6 (1.0462) 189/377
  1.1297    6690.8 (1.0462) 190/377
  1.1383    6687.8 (1.0462) 191/377
  1.1468    6685.2 (1.0462) 192/377
  1.1555    6678.3 (1.0462) 193/377
  1.1641    6675.3 (1.0462) 194/377
  1.1728    6674.0 (1.0462) 195/377
  1.1815    6668.9 (1.0462) 196/377
  1.1902    6664.2 (1.0462) 197/377
  1.1990    6657.4 (1.0462) 198/377
  1.2078    6647.9 (1.0462) 199/377
  1.2167    6643.6 (1.0462) 200/377
  1.2255    6645.0 (1.0462) 201/377
  1.2345    6639.4 (1.0462) 202/377
  1.2434    6639.4 (1.0462) 

...

  1.4893   27709.1 (0.7367) 229/377
  1.4993   28160.7 (0.7367) 230/377
  1.5093   28647.8 (0.7367) 231/377
  1.5193   29206.5 (0.7367) 232/377
  1.5294   29936.2 (0.7367) 233/377
  1.5396   30663.0 (0.7367) 234/377
  1.5498   31437.0 (0.7367) 235/377
  1.5600   32372.8 (0.7367) 236/377
  1.5702   33213.9 (0.7367) 237/377
  1.5805   33918.9 (0.7367) 238/377
  1.5909   34599.7 (0.7367) 239/377
  1.6013   35214.9 (0.7367) 240/377
  1.6117   35850.3 (0.7367) 241/377
  1.6222   36530.5 (0.7367) 242/377
  1.6327   37044.3 (0.7367) 243/377
  1.6432   37405.0 (0.7367) 244/377
  1.6538   37717.1 (0.7367) 245/377
  1.6645   38041.3 (0.7367) 246/377
  1.6751   38482.3 (0.7367) 247/377
  1.6859   38942.3 (0.7367) 248/377
  1.6966   39238.8 (0.7367) 249/377
  1.7074   39584.4 (0.7367) 250/377
  1.7183   40215.0 (0.7367) 251/377
  1.7292   40730.1 (0.7367) 252/377
  1.7401   41083.6 (0.7367) 253/377
  1.7511   41381.6 (0.7367) 254/377
  1.7621   41534.1 (0.7367) 255/377
  1.7732   41822.1 (0.7367) 

  0.7083   74004.3 (0.6998) 6/41
  0.7100   74009.8 (0.6998) 7/41
  0.7118   74001.6 (0.6998) 8/41
  0.7135   73970.2 (0.6998) 9/41
  0.7152   73936.9 (0.6998) 10/41
  0.7169   73876.6 (0.7169) 11/41
  0.7186   73810.5 (0.7186) 12/41
  0.7203   73739.2 (0.7203) 13/41
  0.7221   73659.0 (0.7221) 14/41
  0.7238   73576.2 (0.7238) 15/41
  0.7255   73497.3 (0.7255) 16/41
  0.7272   73417.9 (0.7272) 17/41
  0.7290   73350.5 (0.7290) 18/41
  0.7307   73290.7 (0.7307) 19/41
  0.7324   73253.7 (0.7324) 20/41
  0.7342   73227.9 (0.7342) 21/41
  0.7359   73231.9 (0.7342) 22/41
  0.7376   73254.9 (0.7342) 23/41
  0.7394   73299.5 (0.7342) 24/41
  0.7411   73355.4 (0.7342) 25/41
  0.7428   73423.0 (0.7342) 26/41
  0.7446   73485.9 (0.7342) 27/41
  0.7463   73550.6 (0.7342) 28/41
  0.7481   73599.1 (0.7342) 29/41
  0.7498   73647.8 (0.7342) 30/41
  0.7516   73691.0 (0.7342) 31/41
  0.7533   73720.4 (0.7342) 32/41
  0.7551   73731.8 (0.7342) 33/41
  0.7568   73743.6 (0.7342) 34/41
  0.7586   73745.9

  0.2312   33237.4 (0.1549) 53/377
  0.2361   33267.1 (0.1549) 54/377
  0.2411   33299.4 (0.1549) 55/377
  0.2461   33327.0 (0.1549) 56/377
  0.2511   33349.6 (0.1549) 57/377
  0.2561   33372.0 (0.1549) 58/377
  0.2611   33392.1 (0.1549) 59/377
  0.2662   33415.5 (0.1549) 60/377
  0.2712   33437.8 (0.1549) 61/377
  0.2763   33465.1 (0.1549) 62/377
  0.2815   33490.3 (0.1549) 63/377
  0.2866   33522.6 (0.1549) 64/377
  0.2918   33557.2 (0.1549) 65/377
  0.2969   33588.0 (0.1549) 66/377
  0.3021   33614.3 (0.1549) 67/377
  0.3073   33641.6 (0.1549) 68/377
  0.3126   33665.7 (0.1549) 69/377
  0.3178   33686.0 (0.1549) 70/377
  0.3231   33695.8 (0.1549) 71/377
  0.3284   33694.5 (0.1549) 72/377
  0.3338   33689.5 (0.1549) 73/377
  0.3391   33685.2 (0.1549) 74/377
  0.3445   33682.8 (0.1549) 75/377
  0.3499   33687.6 (0.1549) 76/377
  0.3553   33698.5 (0.1549) 77/377
  0.3607   33710.2 (0.1549) 78/377
  0.3662   33720.9 (0.1549) 79/377
  0.3716   33735.9 (0.1549) 80/377
  0.3771   33749.9 (

  1.6222   31032.4 (1.1990) 242/377
  1.6327   31092.6 (1.1990) 243/377
  1.6432   31185.2 (1.1990) 244/377
  1.6538   31326.4 (1.1990) 245/377
  1.6645   31560.4 (1.1990) 246/377
  1.6751   31752.9 (1.1990) 247/377
  1.6859   31862.0 (1.1990) 248/377
  1.6966   32015.0 (1.1990) 249/377
  1.7074   32167.8 (1.1990) 250/377
  1.7183   32247.1 (1.1990) 251/377
  1.7292   32398.0 (1.1990) 252/377
  1.7401   32528.8 (1.1990) 253/377
  1.7511   32602.1 (1.1990) 254/377
  1.7621   32669.0 (1.1990) 255/377
  1.7732   32691.4 (1.1990) 256/377
  1.7843   32697.0 (1.1990) 257/377
  1.7955   32808.6 (1.1990) 258/377
  1.8067   33159.4 (1.1990) 259/377
  1.8179   33474.3 (1.1990) 260/377
  1.8292   33742.4 (1.1990) 261/377
  1.8406   33824.9 (1.1990) 262/377
  1.8519   33790.4 (1.1990) 263/377
  1.8634   33582.0 (1.1990) 264/377
  1.8748   33706.6 (1.1990) 265/377
  1.8864   33811.4 (1.1990) 266/377
  1.8979   33969.5 (1.1990) 267/377
  1.9096   34113.5 (1.1990) 268/377
  1.9212   34300.9 (1.1990) 

  1.1944 2916438.4 (1.1944) 20/41
  1.1966 2916436.8 (1.1966) 21/41
  1.1988 2916445.4 (1.1966) 22/41
  1.2010 2916474.6 (1.1966) 23/41
  1.2032 2916505.5 (1.1966) 24/41


In [ ]:
good = np.where((np.array(grp.catalog['MAG_AUTO']) > 22) & (np.array(grp.catalog['MAG_AUTO']) < 24))[0]
for g in good:
    id_fit = np.array(grp.catalog['NUMBER'])[g]
    mag_fit = grp.catalog['MAG_AUTO'][g]
    grizli_fit(grp, id_fit = id_fit, field = field,
               use_pz_prior = False, use_phot = True, scale_phot = True,
               templ0 = templ0, templ1 = templ1, ez = ez, ep = ep, pline = pline,)
        